In [5]:
import pandas as pd
import numpy as np
import random

# Definir jugadores con posición y rol
jugadores_info = [
    {"nombre": "Alberto Diaz", "posicion": "Base", "rol": "Defensor de perímetro, organizador"},
    {"nombre": "Alberto Abalde", "posicion": "Escolta", "rol": "Tirador exterior, apoyo en bloqueo"},
    {"nombre": "Joel Parra", "posicion": "Alero", "rol": "3&D, reboteador, defensor de ayudas"},
    {"nombre": "Santi Aldama", "posicion": "Ala-pívot", "rol": "Interior moderno, pick&pop, ayuda en defensa"},
    {"nombre": "Jaime Pradilla", "posicion": "Ala-pívot", "rol": "Jugador físico, bloqueador, cortador"},
    {"nombre": "Sergio de Larrea", "posicion": "Base", "rol": "Creador joven, dirección del ataque"},
    {"nombre": "Dario Brizuela", "posicion": "Escolta", "rol": "Penetrador, generador de puntos, tirador"},
    {"nombre": "Josep Puerto", "posicion": "Alero", "rol": "Tirador de esquina, defensa secundaria"},
    {"nombre": "Juancho Hernangomez", "posicion": "Ala-pívot", "rol": "Versátil, tirador, corre en transición"},
    {"nombre": "Willy Hernangomez", "posicion": "Pívot", "rol": "Juego al poste, pick&roll, rebote ofensivo"}
]

# Zonas de la pista y acciones posibles
zonas_pista = ["perímetro", "zona", "poste bajo", "línea de tres", "esquina", "media distancia", "zona central"]
acciones_ofensivas = ["penetración", "tiro", "pase", "bloqueo directo (con balón)", "bloqueo directo (sin balón)", "corte"]
acciones_defensivas = ["defensa balón", "ayuda", "defensa del bloqueo", "closeout", "rotación"]

tiempo_total = 600  # 10 minutos en segundos

# Probabilidad de que cada jugador esté en acciones de ataque (según rol)
prob_ofensivo = {
    "Alberto Diaz": 0.4,   # Base defensivo -> 40% ataque, 60% defensa
    "Alberto Abalde": 0.7, # Escolta tirador -> 70% acciones ofensivas
    "Joel Parra": 0.5,     # Alero equilibrado
    "Santi Aldama": 0.5,   # Ala-pívot versátil
    "Jaime Pradilla": 0.5, # Ala-pívot físico equilibrado
    "Sergio de Larrea": 0.7,# Base ofensivo
    "Dario Brizuela": 0.8, # Escolta anotador
    "Josep Puerto": 0.6,   # Alero tirador
    "Juancho Hernangomez": 0.7, # Ala-pívot anotador
    "Willy Hernangomez": 0.7    # Pívot ofensivo
}

def generar_datos_jugador(info):
    n = tiempo_total
    # Valores base de HR, velocidad, aceleración según posición
    base_hr = 140 if info["posicion"] in ["Base", "Escolta"] else 135
    base_vel = 5.2 if info["posicion"] in ["Base", "Escolta"] else 4.8  # m/s
    base_acel = 1.7 if info["posicion"] in ["Base", "Escolta"] else 1.4  # m/s^2

    # Generar series con tendencia (fatiga: HR sube, velocidad/aceleración bajan con el tiempo)
    tiempo = np.arange(n)
    hr = np.clip(np.random.normal(base_hr, 8, n) + np.linspace(0, 10, n), 110, 190)
    velocidad = np.abs(np.random.normal(base_vel, 1.0, n) - np.linspace(0, 0.5, n))
    aceleracion = np.abs(np.random.normal(base_acel, 0.4, n) - np.linspace(0, 0.2, n))
    playerload = velocidad * aceleracion  # métrica de carga física instantánea

    # Simular posición GPS en una cancha de 28x15 m (coordenadas acumulativas con ruido)
    x_pos = np.clip(np.cumsum(np.random.normal(0, 0.5, n)), 0, 28)
    y_pos = np.clip(np.cumsum(np.random.normal(0, 0.4, n)), 0, 15)

    df = pd.DataFrame({
        "jugador": info["nombre"],
        "posicion": info["posicion"],
        "tiempo": tiempo,
        "hr": hr,
        "velocidad": velocidad,
        "aceleracion": aceleracion,
        "playerload": playerload,
        "x_pos": x_pos,
        "y_pos": y_pos
    })
    return df

def generar_etiquetas_realistas(info):
    etiquetas = []
    t = 0
    p_off = prob_ofensivo.get(info["nombre"], 0.5)
    while t < tiempo_total - 10:
        dur = random.randint(4, 12)       # duración del evento (4-12 s)
        tipo = "ataque" if random.random() < p_off else "defensa"
        accion = random.choice(acciones_ofensivas) if tipo == "ataque" else random.choice(acciones_defensivas)
        zona = random.choice(zonas_pista)
        # Asignar resultado de la acción (probabilidad de éxito según tipo de acción)
        if tipo == "ataque":
            if accion == "tiro":
                prob = 0.35 if zona in ["línea de tres", "esquina"] else 0.5  # tiro de 3 baja probabilidad
            elif accion == "penetración":
                prob = 0.55  # penetración en zona, prob. moderada de anotar
            elif accion == "pase":
                prob = 0.9   # la mayoría de pases se completan
            elif accion.startswith("bloqueo directo"):
                prob = 0.7   # consideramos ~70% pantalla efectiva
            elif accion == "corte":
                prob = 0.6   # cortes suelen generar canasta ~60%
            else:
                prob = 0.5
        else:  # defensa
            if accion == "defensa balón":
                prob = 0.4   # defensa 1-1 efectiva ~40%
            elif accion == "ayuda":
                prob = 0.5
            elif accion == "defensa del bloqueo":
                prob = 0.5
            elif accion == "closeout":
                prob = 0.4
            elif accion == "rotación":
                prob = 0.5
            else:
                prob = 0.5
        resultado = "Exito" if random.random() < prob else "Fallo"
        etiquetas.append({
            "jugador": info["nombre"],
            "tipo": tipo,
            "accion": accion,
            "zona": zona,
            "inicio": t,
            "fin": min(t + dur, tiempo_total),
            "resultado": resultado
        })
        t += dur + random.randint(1, 5)  # pequeño intervalo aleatorio entre acciones
    return etiquetas

# Generar datos físicos para todos los jugadores
df_fisicos = pd.concat([generar_datos_jugador(j) for j in jugadores_info], ignore_index=True)
# Generar lista de eventos tácticos para todos los jugadores
etiquetas = [evento for jugador in jugadores_info for evento in generar_etiquetas_realistas(jugador)]

# Insertar manualmente eventos combinados específicos (ejemplo de jugadas colectivas)
# Bloqueo indirecto + corte (exitoso)
etiquetas.append({
    "jugador": "Jaime Pradilla", "tipo": "ataque", "accion": "bloqueo directo (sin balón)",
    "zona": "zona central", "inicio": 300, "fin": 307, "resultado": "Exito"
})
etiquetas.append({
    "jugador": "Alberto Abalde", "tipo": "ataque", "accion": "corte",
    "zona": "zona", "inicio": 300, "fin": 305, "resultado": "Exito"
})
# Bloqueo directo + penetración (fallido)
etiquetas.append({
    "jugador": "Santi Aldama", "tipo": "ataque", "accion": "bloqueo directo (con balón)",
    "zona": "zona central", "inicio": 200, "fin": 206, "resultado": "Fallo"
})
etiquetas.append({
    "jugador": "Sergio de Larrea", "tipo": "ataque", "accion": "penetración",
    "zona": "zona", "inicio": 201, "fin": 206, "resultado": "Fallo"
})
# Eliminar cualquier evento aleatorio que se traslape con los insertados, para evitar duplicidad en un mismo jugador/tiempo
for ev in list(etiquetas):
    for esp in ["Jaime Pradilla", "Alberto Abalde", "Santi Aldama", "Sergio de Larrea"]:
        if ev["jugador"] == esp:
            # intervalos de eventos especiales (definidos arriba)
            if (esp == "Jaime Pradilla" or esp == "Alberto Abalde") and not (ev["fin"] < 300 or ev["inicio"] > 307):
                etiquetas.remove(ev)
            if (esp == "Santi Aldama" or esp == "Sergio de Larrea") and not (ev["fin"] < 200 or ev["inicio"] > 206):
                etiquetas.remove(ev)
# Nota: Los eventos específicos se agregaron después de remover los conflictos
# Convertir lista de eventos a DataFrame
df_etiquetas = pd.DataFrame(etiquetas)

# Guardar datasets en CSV
df_fisicos.to_csv("data/datos_fisicos_realistas.csv", index=False)
df_etiquetas.to_csv("data/etiquetas_tacticas_realistas.csv", index=False)
